* [#43](https://github.com/salgo60/SamlaLibris/issues/43)

* [RAA43_linkroot_check.ipynb](https://github.com/salgo60/SamlaLibris/blob/master/notebook/RAA43_linkroot_check.ipynb)

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-12-06 16:04:49


In [2]:
file="./results/links_raa_2025_11_30.csv" # 113 000 poster
file="./results/links_raa_2025_12_01.csv" # 305 000 poster 


In [3]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)



Current Working Directory: /Users/salgo/Documents/GitHub/SamlaLibris/notebook


In [4]:
"""
External Link Health Study – RAA (Sample / HTML)

- Loads links_raa_2025_11_30.csv
- Samples ~300 links for a pilot run
- Measures link availability (HTTP HEAD)
- Detects legacy DSpace XMLUI links
- Generates an HTML report (STATUS_REPORT.html)
"""

import pandas as pd
import requests
import re
import time
from tqdm import tqdm
from urllib.parse import urlparse, quote
from datetime import date

# ------------------------------------------------------------------
# Configuration
# ------------------------------------------------------------------

INPUT_CSV = file
OUTPUT_DIR = "./results"

USER_AGENT = "Wikipedia link health research"
TIMEOUT = 10          # seconds
#DELAY = 0.05          # politeness delay between requests
DELAY = 0          # politeness delay between requests

SAMPLE_SIZE = None     # set to None for full run
RANDOM_STATE = 42     # reproducible sample

# ------------------------------------------------------------------
# Load data + sample
# ------------------------------------------------------------------

df_full = pd.read_csv(INPUT_CSV)

if SAMPLE_SIZE:
    # sample per Wikipedia page to get some spread
    df = (
        df_full
        .drop_duplicates(subset=["Wikipedia-sida", "Extern länk"])
        .groupby("Wikipedia-sida", group_keys=False)
        .apply(lambda x: x.sample(
            min(max(1, SAMPLE_SIZE // max(1, len(df_full["Wikipedia-sida"].unique()))), len(x)),
            random_state=RANDOM_STATE
        ))
        .reset_index(drop=True)
    )

    # If we overshoot a bit, trim to exact SAMPLE_SIZE
    if len(df) > SAMPLE_SIZE:
        df = df.sample(n=SAMPLE_SIZE, random_state=RANDOM_STATE).reset_index(drop=True)
else:
    df = df_full

# ------------------------------------------------------------------
# Helpers
# ------------------------------------------------------------------

def extract_wiki_lang(url):
    try:
        return url.split("//")[1].split(".")[0]
    except Exception:
        return None

def extract_year(url):
    if not isinstance(url, str):
        return None
    m = re.search(r"(19\d{2}|20\d{2})", url)
    return m.group(1) if m else None

def is_legacy_dspace(url):
    if not isinstance(url, str):
        return False
    return "/xmlui/" in url or "/bitstream/handle/" in url

def raa_migration_hint(url):
    if not isinstance(url, str):
        return None
    if "samla.raa.se" not in url:
        return None
    filename = url.split("/")[-1]
    base = re.sub(r"\.pdf$", "", filename, flags=re.I)
    return f"https://samla.raa.se/discover?query={quote(base)}"

def article_label(url):
    """Show a nicer label than the raw Wikipedia URL."""
    if not isinstance(url, str):
        return ""
    if "/wiki/" in url:
        title = url.split("/wiki/")[-1]
        return title.replace("_", " ")
    return url

# measurement (network) ---------------------------------------------

def check_url(url):
    """
    Measure availability of a single URL.

    Returns dict with:
        status: HTTP status or None
        error:  reason for failure (if any)
        elapsed: response time in seconds
    """
    try:
        r = requests.head(
            url,
            allow_redirects=True,
            timeout=TIMEOUT,
            headers={"User-Agent": USER_AGENT}
        )
        time.sleep(DELAY)
        return {
            "status": r.status_code,
            "error": None,
            "elapsed": r.elapsed.total_seconds()
        }

    except requests.exceptions.Timeout:
        time.sleep(DELAY)
        return {"status": None, "error": "timeout", "elapsed": None}

    except requests.exceptions.ConnectionError:
        time.sleep(DELAY)
        return {"status": None, "error": "connection_error", "elapsed": None}

    except requests.exceptions.RequestException as e:
        time.sleep(DELAY)
        return {"status": None, "error": type(e).__name__, "elapsed": None}

# ------------------------------------------------------------------
# Enrich dataframe
# ------------------------------------------------------------------

df["wiki"] = df["Wikipedia-sida"].apply(extract_wiki_lang)
df["year"] = df["Extern länk"].apply(extract_year)
df["legacy_dspace"] = df["Extern länk"].apply(is_legacy_dspace)

# ------------------------------------------------------------------
# Measurement with progress bar
# ------------------------------------------------------------------

tqdm.pandas(desc=f"Measuring {len(df)} external links (sample)")

results = df["Extern länk"].progress_apply(check_url)

df["status"] = results.apply(lambda r: r["status"])
df["error"] = results.apply(lambda r: r["error"])
df["elapsed"] = results.apply(lambda r: r["elapsed"])
df["is_broken"] = df["status"].isin([404, None])

# ------------------------------------------------------------------
# Manual repair candidates (legacy DSpace)
# ------------------------------------------------------------------

manual_repair = df[
    (df["is_broken"]) &
    (df["legacy_dspace"])
].copy()

manual_repair["repair_reason"] = "RAA DSpace XMLUI migration"
manual_repair["migration_hint"] = manual_repair["Extern länk"].apply(raa_migration_hint)

manual_repair.to_csv(
    f"{OUTPUT_DIR}/links_needing_manual_repair_sample.csv",
    index=False
)

# ------------------------------------------------------------------
# Statistics
# ------------------------------------------------------------------

wiki_stats = (
    df.groupby("wiki")
      .agg(
          links_total=("Extern länk", "count"),
          broken_links=("is_broken", "sum"),
          legacy_dspace=("legacy_dspace", "sum")
      )
      .sort_values("broken_links", ascending=False)
)

wiki_stats["broken_pct"] = (
    wiki_stats["broken_links"] / wiki_stats["links_total"] * 100
).round(2)

lang_stats = (
    df.groupby(["lang_code", "lang_name"])
      .agg(
          links=("Extern länk", "count"),
          broken=("is_broken", "sum")
      )
)

lang_stats["broken_pct"] = (
    lang_stats["broken"] / lang_stats["links"] * 100
).round(2)

year_stats = (
    df.dropna(subset=["year"])
      .groupby("year")
      .agg(
          links=("Extern länk", "count"),
          broken=("is_broken", "sum"),
          legacy=("legacy_dspace", "sum")
      )
)

year_stats["broken_pct"] = (
    year_stats["broken"] / year_stats["links"] * 100
).round(2)

domain_health = (
    df.groupby("domain")
      .agg(
          checks=("Extern länk", "count"),
          ok=("status", lambda s: (s == 200).sum()),
          timeouts=("error", lambda e: (e == "timeout").sum()),
          conn_errors=("error", lambda e: (e == "connection_error").sum()),
          mean_latency=("elapsed", "mean")
      )
)

domain_health["problem_ratio"] = (
    (domain_health["timeouts"] + domain_health["conn_errors"]) /
    domain_health["checks"]
).round(2)

domain_health.to_csv(f"{OUTPUT_DIR}/domain_health_sample.csv")

# ------------------------------------------------------------------
# HTML report
# ------------------------------------------------------------------

today = date.today().isoformat()

scope_note = (
    f"This is a <strong>pilot measurement</strong> based on "
    f"{len(df)} sampled links from the full dataset."
    if SAMPLE_SIZE else
    "This measurement covers the full dataset."
)

# prepare nice HTML versions of tables
html_wiki_stats = wiki_stats.head(15).to_html(border=1, classes="table table-sm", justify="left")
html_lang_stats = lang_stats.sort_values("broken", ascending=False).head(15).to_html(border=1, classes="table table-sm", justify="left")
html_year_stats = year_stats.tail(20).to_html(border=1, classes="table table-sm", justify="left")
blocked = domain_health[
    (domain_health["checks"] >= 20) &
    (domain_health["problem_ratio"] > 0.9)
].sort_values("problem_ratio", ascending=False)
html_blocked = blocked.to_html(border=1, classes="table table-sm", justify="left") if len(blocked) else "<p>No clearly blocked domains in this sample.</p>"

# manual repair table – clickable Wikipedia + external links
mr = manual_repair[
    ["Wikipedia-sida", "Extern länk", "repair_reason", "migration_hint"]
].head(25).copy()

mr["Wikipedia-sida"] = mr["Wikipedia-sida"].apply(
    lambda u: f'<a href="{u}">{article_label(u)}</a>'
)
mr["Extern länk"] = mr["Extern länk"].apply(
    lambda u: f'<a href="{u}">{u}</a>'
)
mr["migration_hint"] = mr["migration_hint"].apply(
    lambda u: f'<a href="{u}">{u}</a>' if isinstance(u, str) else ""
)

html_manual_repair = mr.to_html(
    escape=False,
    border=1,
    classes="table table-sm",
    index=False,
    justify="left"
)


Measuring 305250 external links (sample):   2%| | 5785/305250 [33:44<25:29:39,  IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Measuring 305250 external links (sample):  21%|▏| 64211/305250 [6:06:22<26:27:13IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Measuring 305250 external links (sample):  43%|▍| 130934/305250 [16:40:24<22:28:IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashin

In [5]:
broken = df[df["is_broken"]].copy()

broken_display = broken[
    ["Wikipedia-sida", "Extern länk", "status", "error"]
].head(200)

html_broken = broken_display.to_html(
    escape=False,
    border=1,
    classes="table table-sm",
    index=False,
    justify="left"
)


In [6]:


html = f"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<title>External Link Health Study – RAA (Sample)</title>
<style>
body {{
    font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
    margin: 2rem;
    line-height: 1.5;
}}
h1, h2, h3 {{
    font-weight: 600;
}}
.table {{
    border-collapse: collapse;
    margin-top: 0.5rem;
    margin-bottom: 1.5rem;
    font-size: 0.9rem;
}}
.table th, .table td {{
    border: 1px solid #ccc;
    padding: 0.3rem 0.5rem;
}}
.table thead {{
    background: #f2f2f2;
}}
code {{
    background: #f5f5f5;
    padding: 0.1rem 0.3rem;
    border-radius: 3px;
}}
small {{
    color: #666;
}}
</style>
</head>
<body>

<h1>External Link Health Study – RAA (Sample)</h1>
<p><strong>Last updated:</strong> {today}</p>
<p>{scope_note}</p>

<h2>Scope and method</h2>
<p>
This page reports HTTP availability of external links used in Wikipedia,
with a focus on links to Riksantikvarieämbetet and related domains.
It measures <em>technical accessibility</em> of URLs, not the quality of
their content.
</p>

<h2>Summary</h2>
<ul>
  <li><strong>Total links measured:</strong> {len(df):,}</li>
  <li><strong>Broken links:</strong> {df["is_broken"].sum():,} ({df["is_broken"].mean():.1%})</li>
  <li><strong>Legacy DSpace (XMLUI) links:</strong> {df["legacy_dspace"].sum():,}</li>
</ul>

<h2>Broken links by Wikipedia (Top 15)</h2>
{html_wiki_stats}

<h2>Broken links by content language (Top 15)</h2>
{html_lang_stats}

<h2>Link rot over time (year heuristic)</h2>
{html_year_stats}

<h2>Domains showing blocking or throttling (sample-based)</h2>
{html_blocked}


<p><small>
Full sample CSV for this run:
<code>results/links_needing_manual_repair_sample.csv</code><br>
Domain diagnostics:
<code>results/domain_health_sample.csv</code>
</small></p>

</body>
</html>
"""
from datetime import date

today_str = date.today().strftime("%Y_%m_%d")
output_filename = f"STATUS_REPORT_{today_str}.html"

with open(output_filename, "w", encoding="utf-8") as f:
    f.write(html)

print(f"✅ HTML report generated: {output_filename}")
print("✅ Sample CSVs written to ./results/")

✅ HTML report generated: STATUS_REPORT_2025_12_07.html
✅ Sample CSVs written to ./results/


In [ ]:
df["failure_type"] = None

df.loc[df["error"] == "soft_404", "failure_type"] = "soft_404"
df.loc[df["status"] == 404, "failure_type"] = "hard_404"
df.loc[df["error"] == "timeout", "failure_type"] = "timeout"
df.loc[df["error"] == "connection_error", "failure_type"] = "connection_error" 

MANUAL_FAILURES = {"soft_404", "hard_404"} 
all_failures = df[df["is_broken"]].copy()


In [ ]:
all_failures.info()


<class 'pandas.core.frame.DataFrame'>
Index: 11001 entries, 0 to 304453
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Språk           10617 non-null  object 
 1   Wikipedia-sida  11001 non-null  object 
 2   Extern länk     11001 non-null  object 
 3   Wikipedia-länk  11001 non-null  object 
 4   Domän           11001 non-null  object 
 5   lang_code       10617 non-null  object 
 6   lang_name       11001 non-null  object 
 7   domain          11001 non-null  object 
 8   wiki            0 non-null      object 
 9   year            1114 non-null   object 
 10  legacy_dspace   11001 non-null  bool   
 11  status          11001 non-null  float64
 12  error           0 non-null      object 
 13  elapsed         11001 non-null  float64
 14  is_broken       11001 non-null  bool   
 15  failure_type    11001 non-null  object 
dtypes: bool(2), float64(2), object(12)
memory usage: 1.3+ MB


In [15]:
all_failures.to_csv("results/all_failures_export_2025_12_07.csv", index=False)


In [21]:
import pandas as pd

# 1. Filter only failures
errors_df = all_failures[all_failures["is_broken"] == True].copy()

# 2. Drop unwanted columns
errors_df = errors_df.drop(columns=["wiki", "year", "legacy_dspace"], errors="ignore")

# 3. Make URLs clickable
def make_clickable(url):
    if pd.isna(url) or url == "":
        return ""
    return f'<a href="{url}" target="_blank">{url}</a>'

errors_df["Extern länk"] = errors_df["Extern länk"].apply(make_clickable)
errors_df["Wikipedia-länk"] = errors_df["Wikipedia-länk"].apply(make_clickable)

# 4. Generate HTML table
html_table = errors_df.to_html(
    index=False,
    border=0,
    justify="left",
    classes="table table-striped",
    escape=False  # IMPORTANT for clickable URLs
)

# 5. Wrap into full HTML document
html_output = f"""
<html>
<head>
    <meta charset="utf-8">
    <title>Error Report</title>
    <link rel="stylesheet"
          href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
</head>
<body class="p-4">
    <h2>Broken Links / Failed Checks</h2>
    <p>Total failures: {len(errors_df)}</p>
    {html_table}
</body>
</html>
"""

with open("results/errors_report_2025_12_07.html", "w", encoding="utf-8") as f:
    f.write(html_output)


In [13]:
all_failures.head()

,Språk,Wikipedia-sida,Extern länk,Wikipedia-länk,Domän,lang_code,lang_name,domain,wiki,year,legacy_dspace,status,error,elapsed,is_broken,failure_type
0,ace,Leonard Cohen,http://kulturnav.org/language/en/7e908441-ad8d...,https://ace.wikipedia.org/wiki/Leonard_Cohen,kulturnav.org,ace,Acèh,kulturnav.org,None,None,False,404.0,None,0.146233,True,hard_404
1,ace,Agnetha Fältskog,http://kulturnav.org/language/en/1718a400-a2bc...,https://ace.wikipedia.org/wiki/Agnetha_Fältskog,kulturnav.org,ace,Acèh,kulturnav.org,None,None,False,404.0,None,0.175872,True,hard_404
2,ace,Anni-Frid Lyngstad,http://kulturnav.org/language/en/a75e48ae-e70f...,https://ace.wikipedia.org/wiki/Anni-Frid_Lyngstad,kulturnav.org,ace,Acèh,kulturnav.org,None,None,False,404.0,None,0.167199,True,hard_404
3,af,Adils,http://www.raa.se/cms/extern/se_och_besoka/sev...,https://af.wikipedia.org/wiki/Adils,raa.se,af,Afrikaans,www.raa.se,None,None,False,404.0,None,0.212013,True,hard_404
1968,an,Giovanni Pico della Mirandola,https://kulturarvsdata.se/LSH/agents/46681,https://an.wikipedia.org/wiki/Giovanni_Pico_de...,kulturarvsdata.se,an,aragonés,kulturarvsdata.se,None,None,False,404.0,None,0.052075,True,hard_404


In [11]:
print(df["wiki"].value_counts().head())


Series([], Name: count, dtype: int64)


In [12]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-12-07 22:46:22
Total time elapsed: 1841 minutes 32.83 seconds
